## Method 2 Site caller 
(ALPHA version 1)

Author: Zachery Mielko

The script requires the following dependencies:
- Python modules:
    - pandas
    - numpy
    - biopython
- Command line tools:
    - Bedtools


Method 2 takes in the following as **input**:

- Human Genome file (FASTA, must have an index file in the same directory, .fai
    - You can get this from Samtools faidx
- Alignment file from PRIORITY
- Chip-seq peaks (BED file)

The script gives the following as **output**:
- Bed file of centered sites (Centered_PRIORITY.bed)

The output as-is will just be the 1bp center, but you could extract the whole match, which is calculated. 


In [ ]:
################## User defined ###############
# Folder for input/output
IO_Folder = "/Users/ZMielko/Desktop/In_Vivo_Project/Cistrome_Analysis/Ets1/Ets1_PRIORITY/"
# File that has Encode/called peaks
Data_file = 'Ets1DHSv2promoterHg19NoDac.bed'
# Prior assumptions about the kmer PRIORITY alignment
core = [7,13] 
center_pos = 10

################# Common Parameters ###########
# Human genome fasta file
Genome_file = '/Users/ZMielko/Desktop/In_Vivo_Project/Data/human_g1k_v37.fasta'
Overlap_Req = 2
Threshold = 0.45 # Name only for saving
# Output_file_names
Called_TFBS_Save = f"{IO_Folder}/Called_TFBS_{str(Threshold)}" # Called sites
Centered_TFBS_Save = f"{IO_Folder}/Centered_TFBS_{str(Threshold)}.bed" # Centered sites in bed format
Centered_TFBS_Verbose = f"{IO_Folder}/Centered_TFBS_Verbose_{str(Threshold)}.bed" # Centered sites, with scoring and site information
kmer_length = 7
#################################################

###########
# Imports #
###########

import pandas as pd
import numpy as np
import itertools
from Bio import SeqIO
import os
import re
from Bio.Seq import reverse_complement

### kmer prep ###
# Some function requires kmers to be defined. SO this code defines the kmers from the PRIORITY alignment
kmers = pd.read_csv(f'{IO_Folder}/Ets1_45_PRIORITY.txt', sep = '\t')
iscore = []
position = []
kmer = []
for i in kmers['sequences']:
    if '.' in i[core[0]:core[1]]:
        iscore.append(False)
    else:
        iscore.append(True)
    position.append(re.search('(C|G|A|T)',i).start())
    kmer.append(re.findall('[A-Z]{' + str(kmer_length) + '}',i)[0])
kmers['is_core'] = iscore
kmers['kPosition'] = position
kmers['kmer'] = kmer

#############
# Functions #
#############

def DF_fasta(x):
    pos = x[x[0].str.startswith('>')]
    pos = pos.reset_index(drop=True)
    seqs = x[~x[0].str.startswith('>')]
    seqs = seqs.reset_index(drop=True)
    new = pd.DataFrame({'Position':pos[0],'Sequence':seqs[0]})
    return(new)
  
def read_fasta(file):
  records = []
  sequence = []
  with open(file, "rU") as handle:
      for record in SeqIO.parse(handle, "fasta"):
        records.append(record.id)
        sequence.append(str(record.seq))
  Whole_seqs = pd.DataFrame({'Name':records,'Sequence':sequence})
  return Whole_seqs



def kmer_match(String, window,kmer_list, Item):
    """kmer_match takes a string and scans along it to match the score with E-scores"""
    seq_list = []
    pos_list = []
    for i in range(len(String) - (window-1)):
        seq = String[i:i+window]
        pos = i
        if seq in kmer_list:
            seq_list.append(seq)
            pos_list.append(pos)
    result = pd.DataFrame({'Seq':seq_list,'Position':pos_list,'Peak_Seq':Item})
    return(result)


def Score_Blaster(x, kmerSeqs,Thresh=Threshold, kmer_length = kmer_length):
    """Score_Blaster applies kmer_match to a dataframe. Only gives scores above a threshold, zip version"""
    DataFrames = []
    print('Total length: ' + str(len(x)))
    for index,row in enumerate(zip(x['Sequence'], x['Position'])):
      # Counter to keep track of progress
      if int(index)%5000 == 0:
        print(index)
      # kmer match function 
      DataFrames.append(kmer_match(row[0], kmer_length, kmerSeqs,  row[1]))
    result = pd.concat(DataFrames)
    return(result)

def Site_Caller(x):
  """ Siter caller looks for consecutive overlaps """
  Groups = []
  PrevPos = x['Position'][0] -1
  PrevSeq = x['Peak_Seq'][0]
  SeqNumber = 0
  for idx,row in enumerate(zip(x['Peak_Seq'],x['Position'])):
    # When a new Sequence is being read
    if str(row[0]) != str(PrevSeq):
      PrevSeq = str(row[0])
      PrevPos = row[1]
      SeqNumber = SeqNumber + 1
      Groups.append(SeqNumber)
    # When you have a consecutive position
    elif row[1] - PrevPos == 1:
      Groups.append(SeqNumber)
      PrevPos = row[1]
    # When you have a non-consecutive position
    elif row[1] - PrevPos != 1:
      PrevPos = row[1]
      SeqNumber = SeqNumber + 1
      Groups.append(SeqNumber)
  return(Groups)


def Chrom_Splitter(x):
    """Chrom_Splitter takes the concatinated names given in fasta outputs from bedtool's getfasta and turns them into bed compatible columns"""
    Chrom = []
    Start = []
    End = []
    for i in x.Peak_Seq:
        i = i[1:-2]
        cr = i.split(':')
        pos = cr[1].split('-')
        Chrom.append(cr[0])
        Start.append(int(pos[0]))
        End.append(int(pos[1]))
    x['Chromosome'] = Chrom
    x['Start'] = Start
    x['End'] = End
    return(x)

def Genomic_Adjuster(x,orient, kmer_length=kmer_length):
    g = x.groupby(by=['Peak_Seq', 'Site_number'])
    Site_Start = []
    Site_End = []
    Chrom = []
    Center = []
    for name, group in g:
        if orient == '+':
            group_start = int(min(group.Position_x)) + int(group.Start.unique())
            group_end = int(max(group.Position_x)) + int(group.Start.unique()) + kmer_length
            group_center = (center_pos - int(min(group.kPosition))) + int(group.Start.unique()) + int(min(group.Position_x))
            Site_Start.append(group_start)
            Site_End.append(group_end)
            Center.append(group_center)
            Chrom.append(group.Chromosome.unique()[0])
        elif orient == '-':
            slength = int(group.Length.unique())
            group_end = int(slength - min(group.Position_x)) + int(group.Start.unique())
            group_start = int(slength - max(group.Position_x) - kmer_length) + int(group.Start.unique()) 
            group_center = (slength - (center_pos - int(min(group.kPosition)) + int(min(group.Position_x)))) + int(group.Start.unique()) +1
            Site_Start.append(group_start)
            Site_End.append(group_end)
            Center.append(group_center)
            Chrom.append(group.Chromosome.unique()[0])
    result = pd.DataFrame({'Chromosome':Chrom, "Start":Site_Start,
             'End':Site_End, 'Center':Center, 'Orient':orient})
    print(f"Orientation selected is: {orient}")
    return(result)



########################
# Prepare For getfasta #
########################
# Adjust the "Off by 1" issue from bedtools due to the start site being exclusionary
# Bash IO
Bash_Input = IO_Folder + "/Adjusted_peaks"
Bash_Genome = Genome_file
Bash_Out = IO_Folder + '/PeaksWSeqs.fasta'

# Read CSV, filter for short seqs and duplicates, adjust coordinates
Peaks = pd.read_csv(IO_Folder + '/' + Data_file,
                  sep = '\t',
                  header = None,
                  usecols=[0,1,2]) 
Peaks = Peaks[Peaks[2]-Peaks[1] > (kmer_length+Overlap_Req)].drop_duplicates() # filter for short sequences the caller would have trouble with
Peaks[1] = Peaks[1] -1 # Move back 1, adjust for bedtools getfasta
Peaks.to_csv(Bash_Input,
           sep = '\t', index = False, header = None) #Output to getfasta

# Run getFasta
os.system(f'bedtools getfasta -s -fi {Bash_Genome} -bed {Bash_Input} > {Bash_Out} ')

###################
#* Data Analysis *#
###################

print("Reading Files...")

# Read fasta files
PeaksWSeqs = pd.read_csv(Bash_Out, sep = '\t', header=None)
PeaksWSeqs = DF_fasta(PeaksWSeqs)
PeaksWSeqs['Length'] = PeaksWSeqs['Sequence'].apply(lambda x: len(x))
# Make a reverse complement of the sequences
rc_PeaksWSeqs = PeaksWSeqs.copy(deep = True)
rc_PeaksWSeqs['Sequence'] = rc_PeaksWSeqs['Sequence'].apply(lambda x: reverse_complement(x))
# turn the kmers series into a set for fast lookup
chosen_kmers = set(kmers['kmer'])
# Big function of the analysis
def PRIORITY_CALL(Sequences, orientation = '+'):
    Scored = Score_Blaster(x=Sequences, kmerSeqs = chosen_kmers) #Score sites
    Groups = Site_Caller(Scored) # Look for consecutive overlaps (consecutive in genome), groups are a unique value
    Called = Scored.copy(deep = True) # make a copy
    Called['Site_number'] = Groups # add overlap annotation to DF
    print("Finding Overlaps...")
    # Get overlaps of sites with more than 1 consecutive overlap
    values = Called.Site_number.value_counts() 
    Called = Called[Called.Site_number.isin(values.index[values.gt(Overlap_Req-1)])]
    Called['Position'] = Called['Position'].apply(lambda x: int(x))
    # Get the core kmers as a set. Core kmers are those that contain the core range
    core_kmers = set(kmers[kmers['is_core']]['kmer'])
    core_groups = []
    # for each group, only keep it if it has a core kmer
    for group in Called.groupby(by='Site_number'):
        if len(set(group[1].Seq).intersection(core_kmers)) > 0:
            core_groups.append(group[0])
    # Get the core groups as a set, filter by core groups
    core_groups = set(core_groups)
    Called_core = Called[Called['Site_number'].isin(core_groups)]
    # Add kPosition to the DF
    Called2 = pd.merge(Called_core, kmers[['kmer','kPosition']], left_on = 'Seq', right_on='kmer')
    Called2 = Called2.sort_values(by=['Site_number', 'Position'])
    # Check that the kPosition is consecutive
    def checkConsecutive(l): 
        return sorted(l) == l 
    con_groups = []
    for group in Called2.groupby(by='Site_number'):
        if checkConsecutive(list(group[1].kPosition)):
            con_groups.append(group[0])
    con_groups = set(con_groups)
    Called2 = Called2[Called2['Site_number'].isin(con_groups)]
    Called2 = pd.merge(Called2, PeaksWSeqs[['Position','Length']], left_on = 'Peak_Seq', right_on = 'Position')
    Called3 = Chrom_Splitter(Called2)
    final = Genomic_Adjuster(Called3, orient=orientation)
    final = final.drop_duplicates()
    return(final)
# Run the function for positive and negative strands
Positive = PRIORITY_CALL(PeaksWSeqs)
Negative = PRIORITY_CALL(rc_PeaksWSeqs, orientation= '-')
# Merge the results
total = pd.concat([Positive,Negative])
total['Start'] = total['Center']
total['End'] = total['Center']
total = total[['Chromosome', 'Start', 'End', 'Orient']]
total.to_csv(f"{IO_Folder}/Centered_PRIORITY.bed", sep = '\t', header = None, index = False)